In [1]:
!pip install torch torchvision torchaudio



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import pandas as pd
import evaluate
from pathlib import Path

In [3]:
NOTEBOOK_DIR = Path("/".join(__vsc_ipynb_file__.split("/")[:-1]))
print(NOTEBOOK_DIR)

.


In [4]:
# https://huggingface.co/datasets/gretelai/synthetic_text_to_sql
dataset = load_dataset("gretelai/synthetic_text_to_sql")

print("Available data subsets:", dataset.keys())
print("Features: ")
for k, v in dataset["train"][0].items():
    print(f"---> {k:30}: {v}")


Available data subsets: dict_keys(['train', 'test'])
Features: 
---> id                            : 5097
---> domain                        : forestry
---> domain_description            : Comprehensive data on sustainable forest management, timber production, wildlife habitat, and carbon sequestration in forestry.
---> sql_complexity                : single join
---> sql_complexity_description    : only one join (specify inner, outer, cross)
---> sql_task_type                 : analytics and reporting
---> sql_task_type_description     : generating reports, dashboards, and analytical insights
---> sql_prompt                    : What is the total volume of timber sold by each salesperson, sorted by salesperson?
---> sql_context                   : CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_i

In [5]:
train_df = pd.DataFrame(dataset["train"])
test_df = pd.DataFrame(dataset["train"])

In [6]:
from tqdm import tqdm

def check_missing_data(dataset, columns):
    for split in dataset.keys():
        print(f"Checking '{split}' split...")
        for column in tqdm(columns):
            if column not in dataset[split].column_names:
                print(f"  Column '{column}' not found in the dataset!")
                continue
            missing_count = sum(1 for example in dataset[split] if not example[column] or example[column].strip() == "")
            print(f"  {column}: {missing_count} missing or empty entries")
        print()

print("Dataset columns:", dataset["train"].column_names)
check_missing_data(dataset, columns=["sql_prompt", "sql"])

Dataset columns: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation']
Checking 'train' split...


 50%|█████     | 1/2 [00:09<00:09,  9.63s/it]

  sql_prompt: 0 missing or empty entries


100%|██████████| 2/2 [00:19<00:00,  9.69s/it]


  sql: 0 missing or empty entries

Checking 'test' split...


 50%|█████     | 1/2 [00:00<00:00,  1.63it/s]

  sql_prompt: 0 missing or empty entries


100%|██████████| 2/2 [00:01<00:00,  1.72it/s]

  sql: 0 missing or empty entries



In [7]:
def normalize_text(text):
    text = text.lower().strip().replace("\n", " ")
    return text

tokenizer = AutoTokenizer.from_pretrained("t5-large")

In [8]:
def preprocess(example):
    input_text = "Translate to SQL: " + normalize_text(example["sql_prompt"])
    output_text = normalize_text(example["sql"])

    model_inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(output_text, truncation=True, padding="max_length", max_length=128)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_ds = dataset.map(preprocess, remove_columns=dataset["train"].column_names)

available_splits = dataset.keys()
train_ds = tokenized_ds["train"]
validation_ds = tokenized_ds["test"] if "test" in available_splits else None
test_ds = tokenized_ds["test"] if "test" in available_splits else None

In [ ]:
!pip install -r requirements/requirements.txt

^C


Obtaining llamafactory from git+https://github.com/hiyouga/LLaMA-Factory.git@52f25651a2016ddede2283be17cf40c2c1b906ed#egg=llamafactory (from -r requirements/requirements.txt (line 84))
  Skipping because already up-to-date.
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached aiohttp-3.11.18-cp312-cp312-win_amd64.whl.metadata (8.0 kB)
  Using cached airportsdata-20250224-py3-none-any.whl.metadata (9.0 kB)
  

ERROR: Ignored the following versions that require a different python version: 0.36.0 Requires-Python >=3.6,<3.10; 0.37.0 Requires-Python >=3.7,<3.10; 0.38.0 Requires-Python >=3.7,<3.11; 0.38.1 Requires-Python >=3.7,<3.11; 0.52.0 Requires-Python >=3.6,<3.9; 0.52.0rc3 Requires-Python >=3.6,<3.9; 0.53.0 Requires-Python >=3.6,<3.10; 0.53.0rc1.post1 Requires-Python >=3.6,<3.10; 0.53.0rc2 Requires-Python >=3.6,<3.10; 0.53.0rc3 Requires-Python >=3.6,<3.10; 0.53.1 Requires-Python >=3.6,<3.10; 0.54.0 Requires-Python >=3.7,<3.10; 0.54.0rc2 Requires-Python >=3.7,<3.10; 0.54.0rc3 Requires-Python >=3.7,<3.10; 0.54.1 Requires-Python >=3.7,<3.10; 0.55.0 Requires-Python >=3.7,<3.11; 0.55.0rc1 Requires-Python >=3.7,<3.11; 0.55.1 Requires-Python >=3.7,<3.11; 0.55.2 Requires-Python >=3.7,<3.11; 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version t

In [10]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
!pip install "accelerate>=0.26.0"


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-large")

training_args = Seq2SeqTrainingArguments(
    output_dir="./sql_model",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=2,
    predict_with_generate=True,
    logging_dir="./logs"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    tokenizer=tokenizer
)

trainer.train()

C:\Users\szymo\AppData\Local\Temp\ipykernel_13912\3052652965.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
c:\Users\szymo\Desktop\ProgramingStuff\VisualStudioCodeProjects\llm-fine-tuning\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
def evaluate_model(test_dataset):
    metric = evaluate.load("sacrebleu")

    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        decoded_labels = [[label] for label in decoded_labels]
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"bleu": result["score"]}

    results = trainer.evaluate(eval_dataset=test_dataset, compute_metrics=compute_metrics)
    return results

if test_ds:
    test_results = evaluate_model(test_ds)
    print(test_results)

In [ ]:
def driver(question):
    inputs = tokenizer("Translate to SQL: " + question, return_tensors="pt").input_ids
    outputs = model.generate(inputs, max_length=128)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(driver("Find all customers who ordered in 2023"))